# Mark Riley
## DS 710 Final Project
### April 30, 2019

In [1]:
import json
import tweepy
import pandas as pd
import numpy as np
%run twitter_credentials.py

In [2]:
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

In [16]:
# Will attempt to get 1500 tweets with a goal to have at least
# 100 tweets after removing duplicates and tweets by Tesla/Elon
num_needed = 1500

# I will search for the words rela autonomy and exclude retweets
search_terms = ('tesla autonomy -filter:retweets')

tweet_list = [] # Emplty list to hold the search results
last_id = -1 # id of last tweet seen

while len(tweet_list) < num_needed: # Loop until the number of tweets is met
    try:
        # Search tweets
        new_tweets = api.search(q=search_terms, 
                                lang='en', 
                                count = 100, 
                                max_id = str(last_id - 1), 
                                tweet_mode='extended')
    except tweepy.TweepError as e:
        print("Error", e)
        break
    else:
        if not new_tweets:
            print("Could not find any more tweets!")
            break
        
        # Found tweets, add them to the list
        tweet_list.extend(new_tweets)
        
        # Reset the last_id
        last_id = new_tweets[-1].id

# Check to make sure we found enough tweets
len(tweet_list)

1594

In [18]:
# Convert the tweets to a Pandas dataframe starting with the tweet text
data = pd.DataFrame(data=[tweet.full_text for tweet in tweet_list], columns=['Tweets'])

In [19]:
# Get the tweet ID, Date, and Screen Name for each tweet in the list and
# append them as new columns to the data frame
data['ID']   = np.array([tweet.id for tweet in tweet_list])
data['Date'] = np.array([tweet.created_at for tweet in tweet_list])
data['User'] = np.array([tweet.user.screen_name for tweet in tweet_list])

In [24]:
import re # For using regular expressions
from textblob import TextBlob # For determining the sentiment of the tweet using NLTK

In [36]:
def clean_text(row):
    # Remove links, special characters, etc. from the tweet using regex
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", row['Tweets']).split())

def get_tweet_sentiment(row):
    # Create TextBlob object of passed tweet text 
    analysis = TextBlob(row['CleanTweets'])
    
    # Set sentiment based on analysis done by the TextBlob library
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [37]:
# Clean the tweet text
data['CleanTweets'] = data.apply(clean_text, axis = 1)

# Set the sentiment for each tweet
data['Sentiment'] = data.apply(get_tweet_sentiment, axis = 1)

In [45]:
# Drop duplicates from the data frame based on the CleanTweets column
data.drop_duplicates(subset='CleanTweets', keep='first', inplace=True)

1244

In [46]:
data.drop('Tweets', axis = 1, inplace = True)
data.to_csv("tesla_autonomy_tweets.csv", index=False)